#**Zajęcia nr 3**
## Pierwsze ćwiczenia z PULP
Celem zajęć było zapoznanie się z biblioteką o nazwie PULP, która umożliwia rozwiązywanie problemów za pomocą programowania liniowego. Zapoznanie z biblioteką odbyło się na przykładzie problemu *blending problem* umieszczonego na stronie w zakładce Case Studies pod tym [linkiem]( https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html). 

##**Programowanie liniowe**

Programowanie liniowe pozwala na rozwiązywanie problemów optymalizacji poprzez:
1. zapoznanie się z problemem
2. stworzenie odpowiednich zmiennych decyzyjnych
3. utworzenie funkcji celu mającej **postać liniową** oraz określenie, czy należy ją zmaksymalizować czy zminimalizować
4. wprowadzenie warunków ograniczających mających także **postać liniową**
5. wyznaczenie rozwiązania/rozwiązań poprzez zastosowanie algorytmu rozwiązującego problem optymalizacji dla funkcji celu oraz przy zadanych ograniczeniach

Problemy przedstawione na tych oraz kolejnych zajęciach będą rozwiązywane według schematu podanego powyżej.

##**Przykłady**
---
####Instalacja PULP

In [1]:
!pip install pulp

!conda install -c conda-forge pulp


     |████████████████████████████████| 40.6MB 105kB/s 
/bin/bash: conda: command not found


####Test poprawności instalacji

In [ ]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.3.1


In [ ]:
pulp.pulpTestAll()

......

Solver <class 'pulp.apis.coin_api.PULP_CBC_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_DLL'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_PY'> unavailable
Solver <class 'pulp.apis.coin_api.COIN_CMD'> unavailable
Solver <class 'pulp.apis.coin_api.COINMP_DLL'> unavailable
Solver <class 'pulp.apis.glpk_api.GLPK_CMD'> available
Solver <class 'pulp.apis.xpress_api.XPRESS'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI_CMD'> unavailable
Solver <class 'pulp.apis.glpk_api.PYGLPK'> unavailable
Solver <class 'pulp.apis.coin_api.YAPOSIB'> unavailable
Solver <class 'pulp.apis.choco_api.PULP_CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.choco_api.CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.mipcl_api.MIPCL_CMD'> unavailable
Solver <class 'pulp.apis.mosek_api.MOSEK'> unavailable
Solver <class 'pulp.apis.scip_api.SCIP_CMD'> unavailable
	 Testing in

.............

	 Testing continuous LP solution - export solver JSON
	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing the availability of the function pulpTestAll
	 Testing zero subtraction
	 Testing inconsistent lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint


.......

...

........

	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)


..................................

{'a': 53.0, 'b': 45.3, 'c': 459.2}


........
----------------------------------------------------------------------
Ran 86 tests in 1.393s

OK


##**Zadania do rozwiązania**

---

**Zadanie 1**

Zapoznanie się z materiałami dotyczącymi pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejście przez *blending problem*

**Zadanie 2**

Dodanie zmiennej decyzyjnej w blending problem. Stworzenie zmiennej która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

**Zadanie 3**

Dodanie ograniczenia co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

**Zadanie 4**

Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [ ]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

['jabłko:5', 'banan:10']
jabłko:5, banan:10




##**Rozwiązania**

---

**Zadanie 1**

1. Przedstawiony w Case Study *Whiskas Problem* polegał na ustaleniu składu puszki karmy dla kota tak, aby zawartość spełniała odpowiednie wartości odżywcze, a przy tym koszt produkcji puszki będzie jak najmniejszy. <br> 
Podany problem w pierwszej części uproszczono aby skupić się na samej implementacji przy pomocy PULP. <br> W drugiej części przedstawiono pełną specyfikację oraz możliwość łatwiejszego tworzenia dużej ilości zmiennych oraz ograniczeń za pomocą słowników (*dictionary*)


**Część 1 - problem podstawowy**

In [2]:
# pulp import

import pulp
print(pulp.__version__)

2.3.1


In [3]:
# pulp test

pulp.pulpTestAll()

.............

Solver <class 'pulp.apis.coin_api.PULP_CBC_CMD'> available
Solver <class 'pulp.apis.cplex_api.CPLEX_DLL'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_PY'> unavailable
Solver <class 'pulp.apis.coin_api.COIN_CMD'> unavailable
Solver <class 'pulp.apis.coin_api.COINMP_DLL'> unavailable
Solver <class 'pulp.apis.glpk_api.GLPK_CMD'> unavailable
Solver <class 'pulp.apis.xpress_api.XPRESS'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI_CMD'> unavailable
Solver <class 'pulp.apis.glpk_api.PYGLPK'> unavailable
Solver <class 'pulp.apis.coin_api.YAPOSIB'> unavailable
Solver <class 'pulp.apis.choco_api.PULP_CHOCO_CMD'> available
Solver <class 'pulp.apis.choco_api.CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.mipcl_api.MIPCL_CMD'> unavailable
Solver <class 'pulp.apis.mosek_api.MOSEK'> unavailable
Solver <class 'pulp.apis.scip_api.SCIP_CMD'> unavailable
	 Testing inva

.............

	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP solution
	 Testing MIP solution


..............

...

...............

	 Testing elastic constraints (penalty unbounded)


..........................

{'a': 53.0, 'b': 45.3, 'c': 459.2}


.....
----------------------------------------------------------------------
Ran 129 tests in 37.639s

OK


In [4]:
# Import PuLP modeler functions
from pulp import *

In [5]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

In [7]:
# The 2 variables Beef and Chicken are created with a lower limit of zero
x1 = LpVariable("ChickenPercent", 0, None, LpInteger)
x2 = LpVariable("BeefPercent", 0)

In [8]:
# The objective function is added to 'prob' first
prob += 0.013*x1 + 0.008*x2, "Total Cost of Ingredients per can"

In [9]:
# The five constraints are entered
prob += x1 + x2 == 100, "PercentageSum"
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, "ProteinRequirement"
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, "FatRequirement"
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, "FibreRequirement"
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, "SaltRequiement"

In [10]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

[BeefPercent, ChickenPercent]

In [11]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [12]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [13]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

BeefPercent = 66.0
ChickenPercent = 34.0


In [14]:
# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))

Total Cost of Ingredients per can =  0.97


**Część 2 - problem roszerzony**

W tej części sformułowano problem w pełni ze wszystkimi zmiennymi. Do zapisu zmiennych użyto słowników. 

Dane użyte do analizy:

Waga jednej puszki to 100g

Wartości odżywcze produktów w g/100g produktu:

| Stuff      | Protein | Fat   | Fibre | Salt  |
|------------|---------|-------|-------|-------|
| Chicken    | 0.100   | 0.080 | 0.001 | 0.002 |
| Beef       | 0.200   | 0.100 | 0.005 | 0.005 |
| Mutton     | 0.150   | 0.110 | 0.003 | 0.007 |
| Rice       | 0.000   | 0.010 | 0.100 | 0.002 |
| Wheat bran | 0.040   | 0.010 | 0.150 | 0.008 |
| Gel        | 0.000   | 0.000 | 0.000 | 0.000 |



Wartości odżywcze karmy w jednej puszcze 100g:

| Stuff   | Requirement |
|---------|-------------|
| Protein | 8.0         |
| Fat     | 6.0         |
| Fibre   | 2.0         |
| Salt    | 0.4         |

Cena składników w $/g

| Stuff      | Price |
|------------|-------|
| Chicken    | 0.013 |
| Beef       | 0.008 |
| Mutton     | 0.010 |
| Rice       | 0.002 |
| Wheat bran | 0.005 |
| Gel        | 0.001 |

Zmiennymi decyzyjnymi została ilość procentowa 6 składników w puszce. 
Funkcja celu do minimalizacji to koszt puszki obliczany jako suma iloczynów procenta składnika i jego ceny. 

In [19]:
# Import PuLP modeler functions
from pulp import *

In [20]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013,
         'BEEF': 0.008,
         'MUTTON': 0.010,
         'RICE': 0.002,
         'WHEAT': 0.005,
         'GEL': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100,
                  'BEEF': 0.200,
                  'MUTTON': 0.150,
                  'RICE': 0.000,
                  'WHEAT': 0.040,
                  'GEL': 0.000}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000}

In [21]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

In [22]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0)

In [23]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

In [24]:
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"

In [25]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModelv2.lp")

[Ingr_BEEF, Ingr_CHICKEN, Ingr_GEL, Ingr_MUTTON, Ingr_RICE, Ingr_WHEAT]

In [26]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [27]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [28]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0


**Zadanie 2**

Zmienną decyzyjną, jaką postanowiono dodać, to sardynki (sardines). Aby określić wartości odżywcze oraz cenę produktu, posłużono się informacjami zawartymi w Internecie

| Stuff    | Price | Protein | Fat   | Fibre | Salt  |
|----------|-------|---------|-------|-------|-------|
| Sardines | 0.001 | 0.246   | 0.015 | 0.050 | 0.003 |

Następnie przepisano program z zadania 1 części 2, umieszczając w słownikach informacje na temat dodanego produktu.

In [29]:
# Import PuLP modeler functions
from pulp import *

In [30]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013,
         'BEEF': 0.008,
         'MUTTON': 0.010,
         'RICE': 0.002,
         'WHEAT': 0.005,
         'GEL': 0.001,
         'SARDINES': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100,
                  'BEEF': 0.200,
                  'MUTTON': 0.150,
                  'RICE': 0.000,
                  'WHEAT': 0.040,
                  'GEL': 0.000,
                  'SARDINES': 0.246}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.015}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.050}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.003}

In [31]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

In [32]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0)

In [33]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

In [34]:
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"

In [35]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModelv2.lp")

[Ingr_BEEF,
 Ingr_CHICKEN,
 Ingr_GEL,
 Ingr_MUTTON,
 Ingr_RICE,
 Ingr_SARDINES,
 Ingr_WHEAT]

In [36]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [37]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [38]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Ingr_BEEF = 54.822335
Ingr_CHICKEN = 0.0
Ingr_GEL = 10.659898
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_SARDINES = 34.517766
Ingr_WHEAT = 0.0


**Zadanie 3 i 4**

Dodano ograniczenia co do zawartości witaminy B w jednej puszcze 100g pokarmu dla kota. Posłużono się podaną tabelą:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Przy czym uznano, że kot powinien przyjąć co najmniej 0,8 ug witaminy B dziennie (w jednej puszce)



In [39]:
# Import PuLP modeler functions
from pulp import *

In [40]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013,
         'BEEF': 0.008,
         'MUTTON': 0.010,
         'RICE': 0.002,
         'WHEAT': 0.005,
         'GEL': 0.001,
         'SARDINES': 0.001}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100,
                  'BEEF': 0.200,
                  'MUTTON': 0.150,
                  'RICE': 0.000,
                  'WHEAT': 0.040,
                  'GEL': 0.000,
                  'SARDINES': 0.246}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.015}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.050}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.003}

# A dictionary of the vitamin B2 percent in each of the Ingredients is created
vitaminB2Percent = {'CHICKEN': 0.003, 
                    'BEEF': 0.026, 
                    'MUTTON': 0.026, 
                    'RICE': 0.000, 
                    'WHEAT': 0.000, 
                    'GEL': 0.000,
                    'SARDINES': 0.089}

In [41]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

In [42]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0)

In [43]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

In [44]:
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminB2Percent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "VitaminB2Requirement"

In [45]:
# The problem data is written to an .lp file
prob.writeLP("WhiskasModelv2.lp")

[Ingr_BEEF,
 Ingr_CHICKEN,
 Ingr_GEL,
 Ingr_MUTTON,
 Ingr_RICE,
 Ingr_SARDINES,
 Ingr_WHEAT]

In [46]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [47]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [48]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Ingr_BEEF = 54.822335
Ingr_CHICKEN = 0.0
Ingr_GEL = 10.659898
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_SARDINES = 34.517766
Ingr_WHEAT = 0.0


In [55]:
print("Końcowy produkt będzie kosztować ", value(prob.objective), "$, będzie miał w składzie: ")
for v in prob.variables():
    print(v.name, "=", v.varValue, "%")
list_sklad = []
cena_jedn = []
for v in prob.variables():
    list_sklad.append(v.varValue)

for koszt in costs.values():
    cena_jedn.append(koszt)


Końcowy produkt będzie kosztować  0.48375634400000006 $, będzie miał w składzie: 
Ingr_BEEF = 54.822335 %
Ingr_CHICKEN = 0.0 %
Ingr_GEL = 10.659898 %
Ingr_MUTTON = 0.0 %
Ingr_RICE = 0.0 %
Ingr_SARDINES = 34.517766 %
Ingr_WHEAT = 0.0 %
